In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 10)

In [2]:
format_dict = {
               'qty':'{:,}','SHARES':'{:,}','available_qty':'{:,}','volbuy':'{:,}','volsell':'{:,}','volbal':'{:,}',
               'price':'{:,.2f}','PRICE':'{:,.2f}','max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'net':'{:,.2f}','profit':'{:,.2f}','cost':'{:,.2f}','volume':'{:,.2f}','beta':'{:,.2f}',
               'Q4':'{:,.3f}','Q3':'{:,.3f}','Q2':'{:,.3f}','Q1':'{:,.3f}','DIVIDEND':'{:,.3f}',
               'percent':'{:,.2f}%','PERCENT':'{:,.2f}%','yearly':'{:,.2f}%','pct':'{:,.2f}%',
              }

### Tables in the process

In [ ]:
# Sells table in MySQL portfolio database
sql = """
SELECT T.name, T.id AS stock_id, B.date AS buy_date, B.qty, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

### In case of sale, retreive buy id from sells record (buy_id)

In [ ]:
# Buys table in MySQL portfolio database
transaction = 'S'
buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1
names = sells['name']
name = names.to_string(index=False)

sql = """
SELECT T.name, B.stock_id AS stock_id, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

In [ ]:
transaction, buy_id, qty, name

### In case of buy, retreive buy id from latest buy record (id)

In [3]:
sql = """
SELECT T.name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT T.name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,IP,1094,194,2022-01-10,3000,19.2,119.23,8.35,Active,57727.58,DTD,None


In [4]:
transaction = 'B'
buy_ids = buys['id']
buy_id = buy_ids.to_string(index=False)
sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)
names = buys['name']
name = names.to_string(index=False)

transaction, buy_id, qty, name

('B', '1094', 3000, 'IP')

## Begin of Sale transaction

### Begin of Sale process to buy table in MySQL stock database

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

In [ ]:
sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)

In [ ]:
rp = const.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy

### Only when sold out

In [ ]:
sqlDel = '''
DELETE FROM buy
WHERE volbuy = 0'''
rp = const.execute(sqlDel)
rp.rowcount

### End of Sale process to buy table in MySQL stock database

### Begin of Update dividend table in MySQL stock database for sells transactions

In [ ]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update table dividend in mysql stock database for sells transactions

### Begin of Sale process to stocks table in SQLite port_lite database

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

In [ ]:
sell_target = 19.90
sqlUpd = '''
UPDATE stocks
SET available_qty = available_qty + %s,
sell_target = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, sell_target, name)
print(sqlUpd)

In [ ]:
rp = conlite.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

### End of Sale process to stocks table in SQLite port_lite database

## End of Sale process

In [ ]:
## Begin of  Buy process

### Begin of Buy process to buy table in MySQL database

In [7]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'IP'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IP,2021-11-09,7500.0,20.2,0.0,0.0,1,0.185,3,A3


### In case there is the same stock name in port

In [9]:
new_cost = 20.20 # new average cost must be calculated manually beforehand

sqlUpd = '''
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 3000, price = 20.2
WHERE name = "IP"


In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

### In case of totally new stock

In [ ]:
sqlIns = """
INSERT INTO buy 
VALUES('SYNEX','2022-01-07',6000,31.5,0,0,1,0,'3','A2')
"""
print(sqlIns)

In [11]:
#rp = const.execute(sqlIns)
rp.rowcount

1

### End of Buy process to buy table in MySQL database

### Begin of Update dividend table in MySQL stock database for buy transactions

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "IP" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,IP,0.185,0.000,0.000,0.000,0.185,0.00,0.00%,"10,500.0",2022-04-27,2022-04-27,,0


In [16]:
sqlUpd = '''
UPDATE dividend
SET shares = shares + %s
WHERE name = "%s"'''
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 3000
WHERE name = "IP"


In [18]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [19]:
sql = """
SELECT * 
FROM dividend 
WHERE name = "%s" 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = "IP" 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,IP,0.185,0.000,0.000,0.000,0.185,0.00,0.00%,"10,500.0",2022-04-27,2022-04-27,,0


### Begin of Buy process to stocks table in SQLite port_lite database

In [20]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IP'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,665,IP,24.47,10.66,I,19.20,23.20,6.93,0.76,20.20,"3,500",-8,9,"10,500",0,0,15pct,mai


In [23]:
new_cost = 20.20
sell_target = 23.20
status = 'I'
reason = '15pct'

In [24]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, cost = %s, sell_target = 23.20, status = 'I', reason = '15pct'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 3000, cost = 20.2, sell_target = 23.20, status = 'I', reason = '15pct'
WHERE name = 'IP'



In [25]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'IP'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,665,IP,24.47,10.66,I,19.20,23.20,6.93,0.76,20.20,"3,500",-8,9,"10,500",0,0,15pct,mai


### End of Buy process to stocks table in SQLite port_lite database

## End of  Buy process